::::
:::{thebe-button}
:::
::::

# Plot tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`.


In [ ]:
from boilercv_docs.nbs import init

paths = init()

from inspect import Signature

from matplotlib.figure import Figure
from matplotlib.pyplot import subplots
from numpy import diff, gradient, pi
from pandas import DataFrame, Series, merge_ordered, read_hdf
from seaborn import scatterplot

from boilercv.correlations import SYMBOL_LABELS, beta
from boilercv.correlations import nusselt as correlations_nusselt
from boilercv.dimensionless_params import (
    fourier,
    jakob,
    kinematic_viscosity,
    nusselt,
    prandtl,
    reynolds,
    thermal_diffusivity,
)
from boilercv.images import scale_bool
from boilercv_docs.nbs import HIDE
from boilercv_pipeline.experiments.e230920_subcool import (
    GBC,
    OBJECTS,
    THERMAL_DATA,
    TRACKS,
)
from boilercv_pipeline.experiments.e240215_plotting import cat10
from boilercv_pipeline.sets import get_dataset

S = Series

TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""
FIGURES: list[Figure] = []
"""Notebook figures available for export."""

# Track tuning
INITIAL_FRAMES = 50
"""Initial frames to average."""
YPX_SURFACE_THRESHOLD = 400
"""Vertical position of bubble centroids considered attached to the surface."""
YPX_DEPARTURE_THRESHOLD = 420
"""Vertical position of bubble centroids considered to have departed the surface."""
MINIMUM_LIFETIME = 0.010  # (s)
"""Minimum bubble lifetime to consider."""

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

# Plotting
FRAME_INTERVAL = 50
"""Frame interval for plotting."""
CORRELATIONS_PALETTE = cat10
"""For plotting correlations."""
MAX_BETA_MAE = 1
"""Maximum mean absolute error of beta to plot."""
MAX_NUSSELT_MAE = 200
"""Maximum mean absolute error of nusselt to plot."""
TRACKS_ALPHA = 0.3
"""Transparency of the tracks."""
TRACKS_SIZE = 10
"""Size of the tracks."""

HIDE

In [ ]:
path_time = TIME.replace(":", "-")
video = scale_bool(
    get_dataset(path_time, stage="filled", frame=slice(0, None, FRAME_INTERVAL))[
        "video"
    ]
)
frametime = diff(video.time.values).mean() / FRAME_INTERVAL
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # pyright: ignore[reportAssignmentType]
thermal_data = read_hdf(THERMAL_DATA)
subcooling = thermal_data.subcool[TIME]
minimum_frame_lifetime = int(MINIMUM_LIFETIME // frametime)
raw_tracks: DataFrame = read_hdf((TRACKS / f"tracks_{path_time}").with_suffix(".h5"))  # pyright: ignore[reportAssignmentType]


def get_init(ser: S) -> float:
    """Get initial value of a series."""
    return ser.head(minimum_frame_lifetime).mean()


HIDE

## Aligning bubble departures

Exclude bubbles that did not originate from the boiling surface, or that had already departed the surface at the time of recording. Consider a bubble to have departed the surface when its centroid crosses a departure threshold which is about one average bubble diameter above the boiling surface. Define the origin for time of departure for each bubble in this fashion. The resulting time history in **Figure&NonBreakingSpace;2** shows bubble depth, velocity, and diameter for the remainder of its visible lifetime.

Most bubbles rise and collapse at similar rates. Two bubbles rise slower than the rest, but seem to collapse at about the same rate as others.


In [ ]:
long_lived_objects = raw_tracks.query(f"frame_lifetime > {minimum_frame_lifetime}")
departing_long_lived_objects = (
    # Find rows corresponding to stagnant or invalid bubbles
    long_lived_objects.groupby("bubble", **GBC)
    .apply(
        # Don't assign any other columns until invalid rows have been filtered out
        lambda df: df.assign(
            yinit_px=lambda df: df["y_px"].pipe(get_init),
            # Initial y position is close to the surface
            began=lambda df: df["yinit_px"] > YPX_SURFACE_THRESHOLD,
            # When the bubble gets far enough away from the surface
            departed=lambda df: df["y_px"] < YPX_DEPARTURE_THRESHOLD,
        )
    )
    # Filter out invalid rows and drop the columns used to determine validity
    .pipe(lambda df: df[df["began"] & df["departed"]])
    .drop(columns=["began", "departed"])
    # Groupby again after filtering out invalid rows
    .groupby("bubble", **GBC)
    # Now columns that depend on the initial row (*.iat[0]) can be assigned
    .apply(
        lambda df: df.assign(
            frame=lambda df: df["frame"] - df["frame"].pipe(get_init),
            time=lambda df: df["time"] - df["time"].pipe(get_init),
            frame_lifetime=lambda df: df["frame"].iat[-1] - df["frame"].pipe(get_init),
            lifetime=lambda df: df["frame_lifetime"] * frametime,
            y_init=lambda df: df["y"].pipe(get_init),
            x_init=lambda df: df["x"].pipe(get_init),
            init_diameter=lambda df: df["diameter"].pipe(get_init),
            dy_init=lambda df: df["dy"].pipe(get_init),
            dy_init_px=lambda df: df["dy"].pipe(get_init),
            max_diameter=lambda df: df["diameter"].max(),
            diameter_rate_of_change=lambda df: gradient(df["diameter"], df["time"]),
        )
    )
)
liquid_thermal_diffusivity = thermal_diffusivity(
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    density=LIQUID_DENSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
)
liquid_kinematic_viscosity = kinematic_viscosity(
    density=LIQUID_DENSITY, dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY
)
tracks = (
    departing_long_lived_objects.assign(**{
        "Bubble Reynolds number": lambda df: reynolds(
            velocity=abs(df["dy"]),
            characteristic_length=df["diameter"],
            kinematic_viscosity=liquid_kinematic_viscosity,
        ),
        "Initial bubble Reynolds number": (
            lambda df: df.loc[:, ["bubble", "Bubble Reynolds number"]]
            .groupby("bubble", **GBC)
            .transform(get_init)
        ),
        "Bubble Fourier number": fourier(
            initial_bubble_diameter=departing_long_lived_objects["init_diameter"],
            liquid_thermal_diffusivity=thermal_diffusivity(
                thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
                density=LIQUID_DENSITY,
                isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
            ),
            time=departing_long_lived_objects["time"],
        ),
        "Nusselt number": nusselt(
            heat_transfer_coefficient=-(
                2
                * VAPOR_DENSITY
                * LATENT_HEAT_OF_VAPORIZATION
                / subcooling
                * departing_long_lived_objects["diameter_rate_of_change"]
            ),
            characteristic_length=departing_long_lived_objects["diameter"],
            thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        ),
        "Dimensionless bubble diameter": (
            lambda df: df["diameter"] / df["init_diameter"]
        ),
    })
    .rename(columns={v: k for k, v in SYMBOL_LABELS.items()})
    .query("Fo_0 > 0")
)
beta_correlations = beta.get_correlations()
nusselt_correlations = correlations_nusselt.get_correlations()
object_averages = (
    departing_long_lived_objects.set_index("bubble")
    .groupby("bubble", **GBC)
    .mean()
    .mean()
)
constants = {
    "Ja": jakob(
        liquid_density=LIQUID_DENSITY,
        vapor_density=VAPOR_DENSITY,
        liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        subcooling=subcooling,
        latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
    ),
    "Pr": prandtl(
        dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "alpha": thermal_diffusivity(
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
        density=LIQUID_DENSITY,
        isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    ),
    "pi": pi,
}
beta_mae = (
    (
        tracks.loc[:, ["bubble", "Re_b0", "Fo_0", "beta"]]
        .rename(columns={"beta": "experimental"})
        .set_index(["bubble"], drop=True)
    )
    .groupby("bubble", **GBC)
    .apply(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Fo_0": df["Fo_0"],
                                "Re_b0": df["Re_b0"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in beta_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Fo_0"])
            .sum()
        )
        / len(df)
    )
).rename_axis(index="bubble")

beta_mae

In [ ]:
nusselt_mae = (
    (
        tracks.loc[:, ["bubble", "Re_b", "Re_b0", "Fo_0", "beta", "Nu_c"]]
        .rename(columns={"Nu_c": "experimental"})
        .set_index(["bubble"], drop=True)
    )
    .groupby("bubble", **GBC)
    .apply(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Re_b": df["Re_b"],
                                "Re_b0": df["Re_b0"],
                                "Fo_0": df["Fo_0"],
                                "beta": df["beta"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in nusselt_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Re_b", "Fo_0", "beta"])
            .sum()
        )
        / len(df)
    )
    .rename_axis(index="bubble")
)

nusselt_mae

In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
ax.set_xlim(0, MAX_BETA_MAE)
ax.set_ylim(0, MAX_NUSSELT_MAE)
scatterplot(
    ax=ax,
    x="Beta (MAE)",
    y="Nusselt (MAE)",
    hue="Correlation",
    palette=CORRELATIONS_PALETTE,
    data=merge_ordered(
        beta_mae.melt(
            var_name="Correlation", value_name="Beta (MAE)", ignore_index=False
        ),
        nusselt_mae.melt(
            var_name="Correlation", value_name="Nusselt (MAE)", ignore_index=False
        ),
        on=["bubble", "Correlation"],
    ),
)
figure.set_figwidth(1.89 * figure.get_figwidth())
figure.set_figheight(1.30 * figure.get_figheight())
HIDE

In [ ]:
beta_mae_2 = (
    (tracks.loc[:, ["Re_b0", "Fo_0", "beta"]].rename(columns={"beta": "experimental"}))
    .pipe(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Re_b0": df["Re_b0"],
                                "Fo_0": df["Fo_0"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in beta_correlations.items()
            })
            .drop(columns=["experimental", "Re_b0", "Fo_0"])
            .sum()
        )
        / len(df)
    )
    .rename("Beta (MAE)")
)

beta_mae_2

In [ ]:
nusselt_mae_2 = (
    (
        tracks.loc[:, ["Fo_0", "Re_b", "Re_b0", "beta", "Nu_c"]].rename(
            columns={"Nu_c": "experimental"}
        )
    )
    .pipe(
        lambda df: (
            df.assign(**{
                label: (
                    abs(
                        corr.expr(**{
                            kwd: value
                            for kwd, value in {
                                **constants,
                                "Fo_0": df["Fo_0"],
                                "Re_b": df["Re_b"],
                                "Re_b0": df["Re_b0"],
                                "beta": df["beta"],
                            }.items()
                            if kwd in Signature.from_callable(corr.expr).parameters
                        })
                        - df["experimental"]
                    )
                )
                for label, corr in nusselt_correlations.items()
            })
            .drop(columns=["experimental", "Fo_0", "Re_b", "Re_b0", "beta"])
            .sum()
        )
        / len(df)
    )
    .rename("Nusselt (MAE)")
)


nusselt_mae_2

In [ ]:
mae = (
    DataFrame([beta_mae_2, nusselt_mae_2])
    .T.rename_axis(index="Correlation")
    .assign(**{"subcooling": subcooling})
)

mae